### Dealing with imports...

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

### Loading data...

In [2]:
events_df = pd.read_csv('../data/events_up_to_01062018.csv', low_memory=False)
labels_df = pd.read_csv('../data/labels_training_set.csv', low_memory=False)

## Processing

In [3]:
# first save original features to know which have been processed
init_features = list(events_df.columns)

### Dates (timestamp)

In [4]:
# some date processing
def date_proc(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour
    
date_proc(events_df)

Primero hacemos un label encoding con el weekday, luego aplicamos una transfrmacion que contemple la naturaleza ciclica de la semana. Esto ultimo lo aplicaremos tambien al resto de los features ciclicos (como se explica, por ejemplo, aca: https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/)

In [5]:
weekday_le = preprocessing.LabelEncoder()
weekday_le.fit(events_df.weekday)

events_df.weekday = weekday_le.transform(events_df.weekday)

In [6]:
def month_to_cyclic(df):
    df['month_sin'] = df['month'].apply(lambda x: np.sin(2*np.pi*x/12))
    df['month_cos'] = df['month'].apply(lambda x: np.cos(2*np.pi*x/12))
    df.drop('month', axis=1, inplace=True)
    
def day_to_cyclic(df):
    df['day_sin'] = df['day'].apply(lambda x: np.sin(2*np.pi*x/31))
    df['day_cos'] = df['day'].apply(lambda x: np.cos(2*np.pi*x/31))
    df.drop('day', axis=1, inplace=True)

def weekday_to_cyclic(df):
    df['weekday_sin'] = df['weekday'].apply(lambda x: np.sin(2*np.pi*x/7))
    df['weekday_cos'] = df['weekday'].apply(lambda x: np.cos(2*np.pi*x/7))
    df.drop('weekday', axis=1, inplace=True)

def hour_to_cyclic(df):
    df['hour_sin'] = df['hour'].apply(lambda x: np.sin(2*np.pi*x/24))
    df['hour_cos'] = df['hour'].apply(lambda x: np.cos(2*np.pi*x/24))
    df.drop('hour', axis=1, inplace=True)

In [7]:
month_to_cyclic(events_df)
day_to_cyclic(events_df)
weekday_to_cyclic(events_df)
hour_to_cyclic(events_df)

In [8]:
events_df.drop('timestamp', axis=1, inplace=True)

In [9]:
init_features.remove('timestamp')

In [10]:
events_df.person.nunique()

38829

## screen_resolution

In [11]:
def get_screen_width(x):
    if x != "":
        return int(x.split("x")[0])
    else:
        return 0
    
def get_screen_height(x):
    if x != "":
        return int(x.split("x")[1])
    else:
        return 0

def process_screen_res(df):
    df['screen_resolution'].fillna("", inplace=True)
    df['screen_width'] = df['screen_resolution'].apply(lambda x: get_screen_width(x))
    df['screen_height'] = df['screen_resolution'].apply(lambda x: get_screen_height(x))
    df.drop('screen_resolution', axis=1, inplace=True)

In [12]:
process_screen_res(events_df)

In [13]:
init_features.remove('screen_resolution')

## storage

In [14]:
def process_storage_string(x):
    if pd.isna(x):
        return 0
    s = x.split("GB")
    if len(s) == 2:
        # case data in GB
        return int(s[0])
    else:
        # case data in MB
        return int(x.split("MB")[0])/1024

def storage_process(df):
    df.storage = df.storage.apply(lambda x: process_storage_string(x))

In [15]:
storage_process(events_df)

In [16]:
init_features.remove('storage')

## Browser

In [17]:
# veamos valores unicos si cortamos el primer string nomas
new_list=set()
for e in list(events_df.browser_version.unique()):
    if isinstance(e, str):
        new_list.add(e.split()[0])
        
new_list

{'Android',
 'BingPreview',
 'BlackBerry',
 'Chrome',
 'Chromium',
 'Edge',
 'Facebook',
 'Firefox',
 'IE',
 'K-Meleon',
 'Maxthon',
 'Mobile',
 'Opera',
 'Other',
 'Pinterest',
 'Puffin',
 'Safari',
 'Samsung',
 'UC',
 'Vivaldi',
 'WebKit',
 'Yandex'}

Que seran...?
* Android (no es un browser en si)
* K-Meleon
* Maxthon
* Mobile
* Pinterest
* Puffin
* Samsung
* UC
* WebKit
* Yandex

Se confirma que son browsers los siguientes:
* Android (no es un browser en si)
* K-Meleon (http://kmeleonbrowser.org/)
* Maxthon (http://www.maxthon.com/)
* Mobile
* Pinterest
* Puffin (Puffin Browser is a web browser released by CloudMosa for mobile operating systems Android, iOS, Windows and Android TV.)
* Samsung (Samsung Internet for Android is a mobile web browser for smartphones and tablets developed by Samsung. It is based on the open-source Chromium project. It is pre-installed on Samsung Galaxy devices.
* UC (UC Browser is a web browser developed by the Chinese mobile Internet company UCWeb, which is in turn owned by the Alibaba Group. As of August 2018 it is the third most popular mobile browser in the world by market share, after Google Chrome and Safari.)
* WebKit (https://webkit.org/)
* Yandex: un browser (https://browser.yandex.com/)

Nos queda revisar los siguientes:
* Android (no es un browser en si)
* Mobile
* Pinterest

In [18]:
for e in list(events_df.browser_version.unique()):
    if isinstance(e,str):
        if e.lower().split()[0] in ['android','mobile','pinterest']:
            print(e)

Mobile Safari 11
Mobile Safari 9
Android 5.1
Mobile Safari 10
Android 4.4
Android 4.1
Android 4.3
Mobile Safari 8
Android 2.3
Mobile Safari 7
Android 4.0
Android 4.2
Mobile Safari UI/WKWebView 10.2
Mobile Safari UI/WKWebView 10.3
Mobile Safari 10.3
Mobile Safari 11.2
Mobile Safari UI/WKWebView 7.1
Mobile Safari 7.1
Mobile Safari 9.3
Mobile Safari UI/WKWebView 11.2
Mobile Safari 6
Pinterest
Mobile Safari 9.2
Mobile Safari 4.0
Mobile Safari UI/WKWebView 9.3
Mobile Safari 11.1
Mobile Safari 10.1
Android 7
Mobile Safari 11.0
Mobile Safari 7.0
Android 3.2
Mobile Safari UI/WKWebView 11.3
Mobile Safari 5.1


***
O sea que todos los _Mobile_ son `Safari`, todos los _Android_ son meramente eso, Android, y _Pinterest_ aparentemente es un browser tambien (?.

Conclusion: puede quedarse solamente el pirmer string del campo para representar el browser, ya que a nadie le interesa la version del mismo.
***

In [ ]:
def proc_browser(x):
    if isinstance(x, str):
        return x.lower().split()[0]

In [ ]:
events_df.browser_version = events_df.browser_version.apply(lambda x: proc_browser(x) if isinstance(x,str) else x)

## operating_system_version
Se procede analogamente a browser_version

In [ ]:
events_df.operating_system_version.unique()

array([nan, 'Android 5.0.2', 'Ubuntu ', 'Android 7', 'Android 6.0.1',
       'Windows 7 ', 'Windows 10 ', 'iOS 11.0.3', 'Android 6',
       'Android 4.4.4', 'Android 7.1.1', 'Mac OS X 10.12.6',
       'Android 5.1', 'Windows 8.1 ', 'Android 5.1.1', 'Android 8.1',
       'Windows 8 ', 'iOS 9.3.5', 'Android 4.2.2', 'Android 5',
       'iOS 11.3', 'Android 4.1.2', 'Android 4.4.2', 'Android 5.0.1',
       'iOS 11.1.1', 'Windows XP ', 'iOS 10.3.3', 'Windows Phone 8.1',
       'Chrome OS 10452.85', 'Android 8', 'Mac OS X 10.10.4',
       'iOS 11.2.6', 'Android ', 'Android 4.3', 'Mac OS X 10.11.6',
       'Windows Vista ', 'iOS 11.1.2', 'Fedora ', 'Windows Phone 10',
       'Linux ', 'Mac OS X 10.13.4', 'Android 7.1.2', 'iOS 8.1.3',
       'iOS 11.2.1', 'Android 4.0.3', 'FreeBSD ', 'iOS 11.2.2',
       'Android 2.3.6', 'iOS 10.2.1', 'iOS 7.1.2', 'Android 4.0.4',
       'Mac OS X 10.7.5', 'Chrome OS 9901.77', 'Chrome OS 10323.67',
       'Chrome OS 10452.96', 'Other ', 'iOS 8.1.1', 'iOS 11.0.2

In [ ]:
# veamos valores unicos si cortamos el primer string nomas
new_list=set()
for e in list(events_df.operating_system_version.unique()):
    if isinstance(e, str):   # filtro los nan
        new_list.add(e.split()[0])
        
new_list

{'Android',
 'BlackBerry',
 'Chrome',
 'Fedora',
 'FreeBSD',
 'Linux',
 'Mac',
 'Other',
 'Symbian',
 'Tizen',
 'Ubuntu',
 'Windows',
 'iOS'}

In [ ]:
def proc_os(x):
    if isinstance(x, str):
        return x.lower().split()[0]

In [ ]:
events_df.operating_system_version = events_df.operating_system_version.apply(lambda x: proc_os(x) if isinstance(x,str) else x)

## device_type
Se procede analogamente a browser_version

In [ ]:
events_df.device_type.unique()

array([nan, 'Smartphone', 'Computer', 'Tablet', 'Unknown'], dtype=object)

Como son campos de una sola palabra, no tiene sentido hacer lo que se hizo con los anteriores casos.

***

## Limited value categorical features
Lets process features which receive a limited number of values.

### event

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['event'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('event_').reset_index(),
              on='person', how='left')

In [ ]:
events_df.drop('event',axis=1,inplace=True)

In [ ]:
init_features.remove('event')

### staticpage

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['staticpage'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('staticpage_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('staticpage',axis=1,inplace=True)

In [ ]:
init_features.remove('staticpage')

### campaign_source

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['campaign_source'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('campaign_source_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('campaign_source',axis=1,inplace=True)

In [ ]:
init_features.remove('campaign_source')

### search_engine

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['search_engine'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('search_engine_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('search_engine',axis=1,inplace=True)

In [ ]:
init_features.remove('search_engine')

### channel

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['channel'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('channel_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('channel',axis=1,inplace=True)

In [ ]:
init_features.remove('channel')

### new_vs_returning

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['new_vs_returning'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('new_vs_returning_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('new_vs_returning',axis=1,inplace=True)

In [ ]:
init_features.remove('new_vs_returning')

### device_type

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['device_type'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('device_type_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('device_type',axis=1,inplace=True)

In [ ]:
init_features.remove('device_type')

### operating_system_version

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['operating_system_version'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('operating_system_version_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('operating_system_version',axis=1,inplace=True)

In [ ]:
init_features.remove('operating_system_version')

### browser_version

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['browser_version'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('browser_version_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('browser_version',axis=1,inplace=True)

In [ ]:
init_features.remove('browser_version')

### condition

In [ ]:
events_df = events_df.merge(events_df.groupby('person')['condition'].value_counts(normalize=True).unstack(fill_value=0).add_prefix('condition_').reset_index(),
                          on='person', how='left')

In [ ]:
events_df.drop('condition',axis=1,inplace=True)

In [ ]:
init_features.remove('condition')

***

In [ ]:
events_df[['person','model']].groupby('person')[['model']].nunique()['model'].max()

# o sea una misma persona busco como maximo 94 modelos distintos

In [ ]:
events_df[['person','color']].groupby('person')[['color']].nunique()['color'].max()

# o sea una misma persona busco como maximo 43 modelos distintos

In [ ]:
events_df[['person','search_term']].groupby('person')[['search_term']].nunique()['search_term'].max()

# o sea una misma persona busco mediante como maximo 100 terminos distintos

In [ ]:
events_df[['person','city']].groupby('person')[['city']].nunique()['city'].max()

# o sea una misma persona busco desde como maximo 19 ciudades distintas

In [ ]:
events_df[['person','region']].groupby('person')[['region']].nunique()['region'].max()

# o sea una misma persona busco desde como maximo 7 regiones distintas

In [ ]:
events_df[['person','country']].groupby('person')[['country']].nunique()['country'].max()

# o sea una misma persona busco desde como maximo 3 paises distintos

In [ ]:
events_df[['person','url']].groupby('person')[['url']].nunique()['url'].max()

***

In [ ]:
events_df.shape

In [ ]:
labels_df.shape

In [ ]:
events_df.person.nunique()

In [ ]:
labels_df.person.nunique()

## Data partition

In [ ]:
# armo df con registros completos clasificados
train_df = events_df.merge(labels_df, on='person', how='right')

In [ ]:
# armo df con registros a predecir unicamente
to_predict = events_df[~events_df['person'].isin(labels_df.person)].copy()

In [ ]:
train_df.shape

In [ ]:
to_predict.shape

In [ ]:
to_predict.person.nunique()

In [ ]:
labels_df.person.nunique()

In [ ]:
events_df.person.nunique()

In [ ]:
train_df[['person','label']].groupby('person')[['label']].nunique()['label'].unique()

# o sea hay un unico y mismo label para cada persona, que se mantiene igual a lo largo de todos los registros de la persona

***

In [ ]:
to_predict.person.nunique()

In [ ]:
labels_df.person.nunique()

In [ ]:
gr1 = train_df[list(train_df.select_dtypes('object').columns)].fillna("").groupby('person',as_index=False).agg(lambda x: ' '.join(x))

In [ ]:
gr2 = train_df[['person','sku']].groupby('person',as_index=False).max()

In [ ]:
l = list(train_df.select_dtypes(exclude='object').columns)
l.append('person')
l.remove('sku')
gr3 = train_df[l].groupby('person',as_index=False).avg()

In [ ]:
train_df_final = gr1.merge(gr2.merge(gr3))

In [ ]:
gr1 = to_predict[list(to_predict.select_dtypes('object').columns)].fillna("").groupby('person',as_index=False).agg(lambda x: ' '.join(x))

In [ ]:
gr2 = to_predict[['person','sku']].groupby('person',as_index=False).max()

In [ ]:
l = list(to_predict.select_dtypes(exclude='object').columns)
l.append('person')
l.remove('sku')
gr3 = to_predict[l].groupby('person',as_index=False).avg()

In [ ]:
to_predict_final = gr1.merge(gr2.merge(gr3))

In [ ]:
to_predict_final.shape

In [ ]:
train_df_final.shape

In [ ]:
to_predict_final.shape

In [ ]:
train_df_final.shape

# Final export
Uncommenct to save to `.csv`s

In [ ]:
train_df_final.to_csv('../data/train_df_processed_screenResol_storage_dates.csv', index=False)
to_predict_final.to_csv('../data/to_predict_processed_screenResol_storage_dates.csv', index=False)